In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 300

def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])
        
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)

def circularity(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    result = 4 * np.pi * (area / perimeter ** 2)
    return result

def isCircular(contour, hull=False):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    if hull:
        contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    return 0.7 <= circularity <= 1.2
    # return 0.5 <= circularity <= 1.4

In [ ]:
import pickle
blobs = pickle.load(open('MSB_Video_1.blobs', 'rb'))
print(len(blobs))

In [ ]:
circular = [ (fno, blob, pixels) for fno, blob, pixels in blobs if isCircular(blob) ]

In [ ]:
noncircular = [ (fno, blob, pixels) for fno, blob, pixels in blobs if not isCircular(blob) ]
print(len(circular), len(noncircular))

In [ ]:
def plotBlobs(blobs):
    ncols = 25
    nrows = (len(blobs) + ncols - 1) // ncols
    plt.figure(figsize=(ncols/2,nrows/2), dpi=100)
    for i, (fno, blob, pixels) in enumerate(blobs):
        plt.subplot(nrows, ncols, i+1)
        im = cv2.cvtColor(pixels, cv2.COLOR_LAB2RGB)
        plt.imshow(im)
        plt.axis('off')

In [ ]:
plotBlobs(circular)

In [ ]:
plotBlobs(noncircular)

In [ ]:
csorted = sorted(circular, key=lambda t: t[2].shape[1])
#del csorted[13]
#del csorted[12]
#del csorted[0]
#plotBlobs(csorted)

## Write the images for training

In [ ]:
fp = open('positive.dat', 'wt')
for i, (fno, blob, pixels) in enumerate(csorted):
    bim = (255 * isBlue(pixels)).astype(np.uint8)
    fname = 'positive/dot{:03d}.png'.format(i)
    cv2.imwrite(fname, bim)
    print(fname, 1, 2, 2, bim.shape[1]-4, bim.shape[0]-4, file=fp)
fp.close()

## Extract some bad images

Maybe from the same frames? Perhaps they should include some blue? And just make sure they don't overlap any of these?

In [ ]:
vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'
vc = cv2.VideoCapture(vid)
vc.get(cv2.CAP_PROP_FRAME_COUNT), vc.get(cv2.CAP_PROP_FPS), vc.get(cv2.CAP_PROP_FRAME_WIDTH), vc.get(cv2.CAP_PROP_FRAME_HEIGHT)
def grabFrame(fn):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    im = cv2.cvtColor(im.astype(np.float32)/255.0, cv2.COLOR_BGR2LAB)
    return im
show(grabFrame(100))

In [ ]:
import random
negative = []
unused_frames = sorted(set(range(100,3680)) - set(fno for fno, blob, pixels in blobs))
for fno, blob, pixels in csorted:
    f = random.choice(unused_frames)
    frame = grabFrame(f)
    x, y, w, h = cv2.boundingRect(blob)
    x = random.randrange(200,600)
    y = random.randrange(100,400)
    npixels = frame[y-2:y+h+2,x-2:x+w+2,:]
    negative.append((f, blob, npixels))

In [ ]:
plotBlobs(negative)

In [ ]:
fp = open('negative.dat', 'wt')
for i, (fno, blob, pixels) in enumerate(negative):
    bim = (255 * isBlue(pixels)).astype(np.uint8)
    fname = 'negative/dot{:03d}.png'.format(i)
    cv2.imwrite(fname, bim)
    print(fname, file=fp)
fp.close()

In [ ]:
sbad = sorted(noncircular, key=lambda t: max(t[2].shape[0], t[2].shape[1]))
plotBlobs(sbad)
'''    0     1     2    3    4    5    6     7    8     9   10    11   12   13    14   15   16    17   18   19    20   21   22    23   24'''

In [ ]:
del

In [ ]:
nc2 = [ (fno, blob, pixels) for fno, blob, pixels in compressed if not isCircular(blob, True) ]
plotBlobs(nc2)

## Optimize color?

In [ ]:
allPixels = []
for fno, blob, pixels in circular:
    rgb = cv2.cvtColor(pixels, cv2.COLOR_LAB2RGB)
    w, h, _ = np.array(rgb.shape)
    radius = min(w, h) / 2
    ys, xs = np.meshgrid(np.arange(h)-(h-1)/2, np.arange(w)-(w-1)/2)
    inside = (ys**2 + xs**2) < radius**2
    ys, xs = np.where(inside)
    allPixels.append(rgb[ys, xs, :])
allPixels = np.concatenate(allPixels)
allPixels.shape


In [ ]:
avg = allPixels.mean(axis=0)
delta = allPixels - avg
uu, dd, vv = np.linalg.svd(delta, full_matrices = False)
d = vv[0]
vv[0]

In [ ]:
avg

In [ ]:
for fno, blob, pixels in circular[:1]:
    h, w, _ = pixels.shape
    rgb = cv2.cvtColor(pixels, cv2.COLOR_LAB2RGB)
    rgb.shape = (h*w, 3)
    delta = rgb - avg
    u = delta.dot(d)
    nrgb = avg + u[np.newaxis].T * d[np.newaxis]
    nrgb.shape = (h, w, 3)
    plt.imshow(nrgb)

In [ ]:
delta = allPixels - avg
uvals = delta.dot(d)
print(np.min(uvals), np.max(uvals))
errors = delta - uvals[None].T * d
errors = np.sqrt(np.sum(errors**2, axis=1))
ndx = np.argsort(errors)
plt.hist(uvals)
#plt.hist(errors)

In [ ]:
np.std(errors)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def f(x):
    return x

In [ ]:
interact(f, x=10)

In [ ]:
p = noncircular[7][2]


In [ ]:
import PIL.Image
from io import BytesIO
import IPython.display

def toImage(a):
    a = cv2.cvtColor(a, cv2.COLOR_LAB2RGB)
    a = np.uint8(255 * a)
    h, w = a.shape[:2]
    s = 100 / max(w, h)
    w *= s
    h *= s
    img = PIL.Image.fromarray(a,mode='RGB')
    f = BytesIO()
    img.save(f,'png')
    return widgets.Image(
        value=f.getvalue(),
        format='png',
        width=w,
        height=h,
        layout=widgets.Layout(margin='auto')
    )

toImage(p)

In [ ]:
cb = widgets.Checkbox(value=False)
widgets.VBox([toImage(p), cb])

In [ ]:
checks = []
rows = []
row = []
for fno, blob, pixels in noncircular:
    cb = widgets.Checkbox(value=False, description=str(fno), layout=widgets.Layout(width='110px'))
    checks.append(cb)
    im = toImage(pixels)
    vb = widgets.VBox([im, cb])
    row.append(vb)
    if len(row) == 10:
        rows.append(widgets.HBox(row))
        row = []
if len(row) > 0:
    rows.append(widgets.HBox(row))
widgets.VBox(rows)

In [ ]:
[ check.value for check in checks[:10]]

In [ ]:
falseNegative = [ nc for i, nc in enumerate(noncircular) if checks[i].value ]

In [ ]:
len(falseNegative)

In [ ]:
good = csorted.copy()
good.extend(falseNegative)
len(good)

In [ ]:
import pickle
pickle.dump(good, open('MSB_Video_1_positive.blobs', 'wb'))

In [ ]:
# get the area of the good ones
areas = sorted([ cv2.contourArea(blob) for fno, blob, pixels in good ])

In [ ]:
areas[0], areas[-1]

In [ ]:
trueNegative = [ nc for i, nc in enumerate(noncircular) if not checks[i].value ]
csorted = sorted(circular, key=lambda t: t[2].shape[1])
falsePositive = csorted[12:14]
bad = trueNegative.copy()
bad.extend(falsePositive)

# filter by area so we don't have those huges ones
fbad = [ b for b in bad if cv2.contourArea(b[1]) < 1500 ]
len(bad), len(fbad)

In [ ]:
pickle.dump(fbad, open('MSB_Video_1_negative.blobs', 'wb'))

In [ ]:
2

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport ImageChooser
from ImageChooser import ImageChooser
chooser = ImageChooser()
chooser.show([pixels for fno, blob, pixels in fbad[:10]])


In [ ]:
c,r = chooser.extract()
len(c), len(r)